In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('src'), '..')))

import pandas as pd
from sklearn.model_selection import train_test_split
import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
generator = torch.Generator()
generator.manual_seed(seed_value)
torch.backends.cudnn.deterministic = True

from functools import partial



from src.tools import upsampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
BATCH_SIZE = 1024
INIT_PARAM = 512
NUM_FEATURES = 50
GAMMA = 0.35

In [3]:
df = pd.read_csv('../data/data_fe.csv')
feature_importance = pd.read_csv('../data/feature_importance/feature_imporstance_WS.csv')

In [4]:
features = list(feature_importance.iloc[:NUM_FEATURES]['feature_names'])
df = df[features + ['Machine failure']]

df.to_csv('../data/df_to_enc.csv', index = False)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Machine failure']),
                                                 df['Machine failure'],
                                                 shuffle=True,
                                                 stratify=df['Machine failure'], random_state=42,
                                                 train_size=0.7)

In [6]:
from src.imbalanced_learing.data_level import data_level

In [7]:
dl = data_level(X_train, y_train)
data = dl.SMOTE(0.5)

In [12]:
pd.DataFrame(data)[50].value_counts()

0.0    6763
1.0    3381
Name: 50, dtype: int64

In [6]:
X_train, y_train = upsampling(GAMMA, X_train, y_train)

In [7]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis = 1)

train_dataset = TableDatasetDF(df_train)
val_dataset = TableDatasetDF(df_test)

In [8]:
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

val_dl = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    generator=generator
)

In [9]:
model = Baseline_classifier(NUM_FEATURES, INIT_PARAM)

In [10]:
pytorch_total_params = sum(p.numel() for p in model.parameters())

In [11]:
pytorch_total_params

730834

In [12]:
loss = Loss_class(FocalLoss(gamma=2))
model_factory = partial(Model_class)
optimizer_factory = partial(torch.optim.AdamW)
scheduler_factory = partial(lr.ExponentialLR)

model_params = dict(model=model,
                    device=device)

optimizer_params = dict(weight_decay=1e-3, lr=1e-2)
scheduler_params = dict(gamma=0.90)

learning_params = dict(batch_size=BATCH_SIZE, num_epoch=40)

wandb_init_params = dict(
    name=f'BL_{INIT_PARAM}_{BATCH_SIZE}_{NUM_FEATURES}_{GAMMA}',
    project="Internship_project",
    dir = '../logs/'
)

In [13]:
trainer = Trainer_classifier(train_dl,
                  val_dl,
                  loss,
                  model_factory=model_factory,
                  optimizer_factory=optimizer_factory,
                  scheduler_factory=scheduler_factory,
                  model_params=model_params,
                  optimizer_params=optimizer_params,
                  scheduler_params=scheduler_params,
                  log=True,
                  wandb_init_params=wandb_init_params,
                  model_dir='../logs/nn_models/classifier/',
                  saving_model=False
                  )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmitrii_fomin (dmitrii_fomin_uga). Use `wandb login --relogin` to force relogin


In [14]:
trainer.train_model(learning_params)
wandb.finish()

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 81.11it/s]


Epoch: 1 of 40, 0.029 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 78.84it/s]


Epoch: 2 of 40, 0.003 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 78.80it/s]


...

Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 77.37it/s]


Epoch: 39 of 40, 0.005 min


Let's see how good I am...: 100%|██████████| 3/3 [00:00<00:00, 74.06it/s]

Epoch: 40 of 40, 0.003 min


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,█▇▇▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy,▁▄▆▆▇▇▇▇▇▇██▇▇▇▇████████████████████████
test_auc_score,▁▅▅▆▃▁▆▇█▆█▆▆▇█▇█████▇████████████▇▇███▇
test_f1_score,▁▄▅▆▇▆▆▇▇▇█▇▇▇▇▇█▇████▇▇████████████████
test_fpr,█▅▃▃▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▆▃▃▂▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_tpr,█▇▇▇▅▆▆▄▆▄▆▂▄▄▆▆▅▂▂▂▃▃▃▂▃▁▂▁▂▁▃▃▃▂▂▁▁▁▂▂
train_accuracy,▁▅▆▆▆▅▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇███▇█▇██▇█▇████
train_auc_score,▁▇███▄▇▇███▇█▇█▇██▇█▇▇███████▇███▇██████
train_f1_score,▁▅▆▆▆▄▅▆▆▆▇▆▆▇▇▆▇▇▇▇▇▇▇▇▇███▇█▇██▇█▇████
